In [1]:
#🚀 PRIMERA ETAPA: Configuración y exploración inicial
#1️⃣ Crear la estructura de carpetas (si aún no existe)

#3️⃣ Instalar librerías necesarias
#En la terminal:
#pip install pandas numpy matplotlib seaborn plotly openpyxl jupyter scikit-learn python-dateutil


# --- Mi Minimarket: Análisis de datos inicial ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from dateutil import parser

# Configuración visual
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8')

# --- 1. Carga de datos ---
clientes = pd.read_excel('database/clientes.xlsx')
productos = pd.read_excel('database/productos.xlsx')
ventas = pd.read_excel('database/ventas.xlsx')
detalle = pd.read_excel('database/detalle_ventas.xlsx')

# --- 2. Inspección rápida ---
print("Clientes:")
display(clientes.head())

print("\nProductos:")
display(productos.head())

print("\nVentas:")
display(ventas.head())

print("\nDetalle Ventas:")
display(detalle.head())


Clientes:


,id_cliente,nombre_cliente,email,ciudad,fecha_alta
0,1,Mariana Lopez,mariana.lopez@mail.com,Carlos Paz,2023-01-01
1,2,Nicolas Rojas,nicolas.rojas@mail.com,Carlos Paz,2023-01-02
2,3,Hernan Martinez,hernan.martinez@mail.com,Rio Cuarto,2023-01-03
3,4,Uma Martinez,uma.martinez@mail.com,Carlos Paz,2023-01-04
4,5,Agustina Flores,agustina.flores@mail.com,Cordoba,2023-01-05



Productos:


,id_producto,nombre_producto,categoria,precio_unitario
0,1,Coca Cola 1.5L,Alimentos,2347
1,2,Pepsi 1.5L,Limpieza,4973
2,3,Sprite 1.5L,Alimentos,4964
3,4,Fanta Naranja 1.5L,Limpieza,2033
4,5,Agua Mineral 500ml,Alimentos,4777



Ventas:


,id_venta,fecha,id_cliente,nombre_cliente,email,medio_pago
0,1,2024-06-19,62,Guadalupe Romero,guadalupe.romero@mail.com,tarjeta
1,2,2024-03-17,49,Olivia Gomez,olivia.gomez@mail.com,qr
2,3,2024-01-13,20,Tomas Acosta,tomas.acosta@mail.com,tarjeta
3,4,2024-02-27,36,Martina Molina,martina.molina@mail.com,transferencia
4,5,2024-06-11,56,Bruno Diaz,bruno.diaz@mail.com,tarjeta



Detalle Ventas:


,id_venta,id_producto,nombre_producto,cantidad,precio_unitario,importe
0,1,90,Toallas Húmedas x50,1,2902,2902
1,2,82,Aceitunas Negras 200g,5,2394,11970
2,2,39,Helado Vainilla 1L,5,469,2345
3,2,70,Fernet 750ml,2,4061,8122
4,2,22,Medialunas de Manteca,1,2069,2069


In [2]:
#🧹 FASE 2: Limpieza y Transformación (ETL con Pandas)
#🎯 Objetivo

#Verificar la estructura, tipos de datos, valores nulos y duplicados.

#Estandarizar formatos (fechas, texto, numéricos).

#Integrar las 4 tablas (clientes, productos, ventas, detalle_ventas) en una sola vista analítica.

# --- 3. Inspección de estructura ---
for nombre, df in [('Clientes', clientes), ('Productos', productos), ('Ventas', ventas), ('Detalle Ventas', detalle)]:
    print(f"\n===== {nombre.upper()} =====")
    print(df.info())
    print(f"Duplicados: {df.duplicated().sum()}")
    print(f"Valores nulos:\n{df.isnull().sum()}")



===== CLIENTES =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id_cliente      100 non-null    int64         
 1   nombre_cliente  100 non-null    object        
 2   email           100 non-null    object        
 3   ciudad          100 non-null    object        
 4   fecha_alta      100 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 4.0+ KB
None
Duplicados: 0
Valores nulos:
id_cliente        0
nombre_cliente    0
email             0
ciudad            0
fecha_alta        0
dtype: int64

===== PRODUCTOS =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id_producto      100 non-null    int64 
 1   nombre_producto  100 non-null

In [3]:
#Vamos a realizar un proceso profesional y controlado de recategorización de productos en el dataset productos, de modo que puedas detectar inconsistencias, analizarlas y corregirlas sin perder trazabilidad.

#🔍 Qué observarás:

#Errores comunes: mayúsculas/minúsculas, tildes, espacios extra, nombres duplicados (ej: “Lácteos”, “lacteos”, “Lacteos”).

#Productos mal ubicados o genéricos (ej: “Sin categoría”, “Otros”, “Default”).

# --- Análisis de categorías ---
print("Categorías únicas:")
print(productos['categoria'].unique())

print("\nConteo de productos por categoría:")
display(productos['categoria'].value_counts())


Categorías únicas:
['Alimentos' 'Limpieza']

Conteo de productos por categoría:


categoria
Alimentos    50
Limpieza     50
Name: count, dtype: int64

In [4]:
#Listar productos sospechosos por categoría

#Si detectas categorías con pocos registros o nombres raros, inspecciónalas:

# --- Ver productos por categoría específica ---
categoria = "otros"  # reemplaza por la categoría sospechosa
display(productos[productos['categoria'].str.lower() == categoria.lower()])


,id_producto,nombre_producto,categoria,precio_unitario


In [ ]:
#Normalizar texto antes de recategorizar

#Conviene unificar el formato para que no existan variantes de texto:

# --- Limpieza de texto en categorías ---
productos['categoria'] = productos['categoria'].str.strip().str.title()

# Correcciones básicas automáticas
productos['categoria'] = productos['categoria'].replace({
    'Lacteos': 'Lácteos',
    'Lacteo': 'Lácteos',
    'Verdura': 'Verduras',
    'Fruta': 'Frutas',
    'Cereal': 'Cereales',
    'Otros': 'Otros Productos',
    'Alimento': 'Alimentos'
})


In [ ]:
#Bloque automatizado de sugerencias de recategorización

#Este bloque agrupa los productos según coincidencias con palabras clave comunes y te muestra sugerencias para corregirlos.

# --- Sugerencias de recategorización automática ---

# Diccionario de palabras clave → categoría sugerida
reglas_categoria = {
    "Jugo|Bebida|Agua|Refresco|Gaseosa|Té|Energética|Mate|Pepsi|Cerveza|Avena|Vino|Ron|Whisky|Fernet|Cola|Sprite|Licor|Vodka": "Bebidas",
    "Manzana|Banano|Naranja|Pera|Uva": "Bebidas",
    "Pan|Ponqué|Bizcocho|Panela|Mermelada|Manteca": "Panadería",
    "Yogur|Leche|Queso|Mantequilla": "Lácteos",
    "Arroz|Frijol|Lenteja|Cereal|Frutos secos|Garbanzos|Granola": "Granos y Cereales",
    "Tomate|Cebolla|Papa|Lechuga|Zanahoria|Verduras": "Verduras",
    "Detergente|Jabón|Limpiador|Desinfectante|Lacandina|Shampoo|Servilletas|Cepillo|Mascarilla|Limpiavidrios|Esponjas|Desodorante": "Limpieza",
    "Galleta|Chocolate|Dulce|Confite|Alfajor|Maní|Turrón|Azúcar|Caramelo|Chupetín|Stevia|Pizza|Helado|Galletitas|Chicle Menta": "Snacks y Dulces"
}

# Buscar productos candidatos para recategorización
sugerencias = []

for patron, categoria_sugerida in reglas_categoria.items():
    mask = productos['nombre_producto'].str.contains(patron, case=False, na=False)
    df_sugerido = productos.loc[mask & (productos['categoria'] != categoria_sugerida), 
                                ['id_producto', 'nombre_producto', 'categoria']]
    if not df_sugerido.empty:
        df_sugerido['categoria_sugerida'] = categoria_sugerida
        sugerencias.append(df_sugerido)

# Concatenar resultados
if sugerencias:
    sugerencias_df = pd.concat(sugerencias, ignore_index=True)
    print("🔍 Productos potencialmente mal categorizados (según palabra clave):")
    display(sugerencias_df)
else:
    print("✅ No se detectaron productos fuera de su categoría esperada.")


🔍 Productos potencialmente mal categorizados (según palabra clave):


,id_producto,nombre_producto,categoria,categoria_sugerida
0,33,Chocolate con Leche 100g,Snacks y Dulces,Bebidas
1,43,Salsa de Tomate 500g,Verduras,Bebidas
2,33,Chocolate con Leche 100g,Snacks y Dulces,Lácteos
3,36,Dulce de Leche 400g,Snacks y Dulces,Lácteos
4,93,Cepillo de Dientes,Alimentos,Limpieza
5,95,Mascarilla Capilar,Alimentos,Limpieza
6,97,Limpiavidrios 500ml,Alimentos,Limpieza
7,99,Esponjas x3,Alimentos,Limpieza
8,24,Galletitas Chocolate,Bebidas,Snacks y Dulces
9,32,Chocolate Amargo 100g,Bebidas,Snacks y Dulces


In [21]:
#Aplicar todas las sugerencias automáticamente

#Si revisaste sugerencias_df y estás de acuerdo con todas las correcciones sugeridas:

# --- Aplicar todas las sugerencias automáticamente ---
for _, fila in sugerencias_df.iterrows():
    productos.loc[productos['id_producto'] == fila['id_producto'], 'categoria'] = fila['categoria_sugerida']

# Guardar los cambios en un nuevo archivo
#productos.to_excel('database/productos_actualizados.xlsx', index=False)
#print("💾 Todas las sugerencias aplicadas y guardadas en 'productos_actualizados.xlsx'.")


In [8]:
#Recategorización manual controlada

#A veces necesitarás reasignar producto por producto.
#Por ejemplo, si el dataset tiene un producto “Yogur Natural” en categoría “Snacks”, pero debería ir en “Lácteos”:

# --- Corrección manual ---

#Puedes crear varias reglas así:
#productos.loc[productos['nombre_producto'].str.contains("Pan", case=False), 'categoria'] = "Panadería"
#productos.loc[productos['nombre_producto'].str.contains("Jugo|Bebida|Agua|Fanta|Energética|Mate|Té|Leche|Queso", case=False), 'categoria'] = "Alimentos"



In [22]:
#Verificar los cambios

#Después de hacer tus correcciones:

print("Categorías finales actualizadas:")
display(productos['categoria'].value_counts())

# Opcional: vista previa por categoría
for cat in productos['categoria'].unique():
    subset = productos[productos['categoria'] == cat].head(20)
    print(f"\n{cat}:")
    display(subset[['id_producto', 'nombre_producto']])


Categorías finales actualizadas:


categoria
Bebidas              25
Snacks y Dulces      20
Limpieza             17
Alimentos            12
Lácteos               9
Panadería             8
Granos y Cereales     6
Verduras              3
Name: count, dtype: int64


Bebidas:


,id_producto,nombre_producto
0,1,Coca Cola 1.5L
1,2,Pepsi 1.5L
2,3,Sprite 1.5L
3,4,Fanta Naranja 1.5L
4,5,Agua Mineral 500ml
5,6,Jugo de Naranja 1L
6,7,Jugo de Manzana 1L
7,8,Energética Nitro 500ml
8,9,Yerba Mate Suave 1kg
9,10,Yerba Mate Intensa 1kg



Alimentos:


...

,id_producto,nombre_producto
27,28,Papas Fritas Clásicas 100g
28,29,Papas Fritas Onduladas 100g
77,78,Verduras Congeladas Mix



Granos y Cereales:


,id_producto,nombre_producto
30,31,Mix de Frutos Secos 200g
34,35,Barrita de Cereal 30g
43,44,Arroz Largo Fino 1kg
45,46,Lentejas Secas 500g
46,47,Garbanzos 500g
62,63,Granola 250g



Limpieza:


,id_producto,nombre_producto
41,42,Vinagre de Alcohol 500ml
47,48,Porotos Negros 500g
51,52,Detergente Líquido 750ml
53,54,Jabón de Tocador
54,55,Shampoo 400ml
55,56,Papel Higiénico x4
56,57,Servilletas x100
89,90,Toallas Húmedas x50
91,92,Crema Dental 90g
92,93,Cepillo de Dientes
